In [2]:
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col, collect_list, udf
from collections import defaultdict
import numpy as np

from pyspark.sql.session import SparkSession
from pyspark import SparkContext
import dill
import os.path

In [3]:
# spark = SparkSession.builder \
#     .appName("concrec-recall") \
#     .master("local[*]") \
#     .config("spark.driver.bindAddress", "127.0.0.1") \
#     .config("spark.ui.port", "4050") \
#     .getOrCreate()
spark = SparkSession.builder.appName("concrec-recall").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/21 21:04:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
BASE_PATH = '../dataset/'
## DataFrame
rating_df = spark.read.csv(BASE_PATH + 'rating.csv', header=True, inferSchema=True)
rating_df.printSchema()
rating_df = rating_df.where('rating > 7')

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [5]:
## user_id <=> [anime_id] group by user_id
watch_seq_df = rating_df \
    .groupBy('user_id') \
    .agg(
        collect_list(col('anime_id').cast('string')).alias('anime_ids') \
    )

In [6]:
## bring it to the driver node as a Python list.
watch_seq = watch_seq_df.collect()

In [7]:
watch_seq[1]

Row(user_id=27, anime_ids=['356', '1195', '1535', '1575', '1604', '1840', '2167', '2904', '2993', '3455', '3712', '4181', '4214', '4224', '4654', '6201', '6213', '6324', '6347', '6547', '6702', '7054', '7592', '7593', '8074', '8516', '8525', '8536', '8769', '8841', '8937', '9041', '9513', '9919', '10049', '10080', '10213', '10620', '10719', '10790', '10793', '11319', '11617', '11633', '11703', '11743', '11757', '11933', '12049', '13659', '13759', '14075', '14227', '14345', '14467', '14741', '14749', '14813', '14967', '15225', '15315', '15451', '15583', '15809', '16049', '16157', '16444', '16498', '16512', '16668', '16706', '17651', '17741', '18857'])

In [8]:
watch_seq = [r['anime_ids'] for r in watch_seq]
# watch_seq[1]

In [9]:
matrix = defaultdict(lambda: defaultdict(int)) ## matrix['a']['b'] += 1

In [10]:
def add_seq_to_matrix(seq, matrix):     ## matrix[a][b]++ and matrix[b][a]++
    for i in range(len(seq)):
        for j in range(i + 1, len(seq)):
            a = seq[i]
            b = seq[j]
            if a == b:
                continue
            matrix[a][b] += 1
            matrix[b][a] += 1

In [11]:
add_seq_to_matrix(watch_seq[1], matrix)
# print(matrix)

In [12]:
for i in range(len(watch_seq)):
    if i % 1000 == 0:
        print(f'matrix add seq {i}')
    seq = watch_seq[i]
    add_seq_to_matrix(seq, matrix)

matrix add seq 0
matrix add seq 1000
matrix add seq 2000
matrix add seq 3000
matrix add seq 4000
matrix add seq 5000
matrix add seq 6000
matrix add seq 7000
matrix add seq 8000
matrix add seq 9000
matrix add seq 10000
matrix add seq 11000
matrix add seq 12000
matrix add seq 13000
matrix add seq 14000
matrix add seq 15000
matrix add seq 16000
matrix add seq 17000
matrix add seq 18000
matrix add seq 19000
matrix add seq 20000
matrix add seq 21000
matrix add seq 22000
matrix add seq 23000
matrix add seq 24000
matrix add seq 25000
matrix add seq 26000
matrix add seq 27000
matrix add seq 28000
matrix add seq 29000
matrix add seq 30000
matrix add seq 31000
matrix add seq 32000
matrix add seq 33000
matrix add seq 34000
matrix add seq 35000
matrix add seq 36000
matrix add seq 37000
matrix add seq 38000
matrix add seq 39000
matrix add seq 40000
matrix add seq 41000
matrix add seq 42000
matrix add seq 43000
matrix add seq 44000
matrix add seq 45000
matrix add seq 46000
matrix add seq 47000
matri

In [14]:
print(len(watch_seq))

69002


In [15]:
print(matrix['356'])

defaultdict(<class 'int'>, {'1195': 2582, '1535': 4730, '1575': 4879, '1604': 1196, '1840': 2274, '2167': 3567, '2904': 4479, '2993': 1693, '3455': 1227, '3712': 1817, '4181': 3165, '4214': 1337, '4224': 3668, '4654': 2309, '6201': 737, '6213': 1551, '6324': 712, '6347': 1976, '6547': 3965, '6702': 1876, '7054': 2023, '7592': 896, '7593': 971, '8074': 2753, '8516': 1368, '8525': 1952, '8536': 424, '8769': 1912, '8841': 1829, '8937': 1625, '9041': 1437, '9513': 1012, '9919': 2586, '10049': 793, '10080': 1725, '10213': 543, '10620': 2768, '10719': 1782, '10790': 1288, '10793': 2402, '11319': 1186, '11617': 1788, '11633': 1064, '11703': 533, '11743': 477, '11757': 3632, '11933': 988, '12049': 470, '13659': 1110, '13759': 1828, '14075': 1468, '14227': 1420, '14345': 1683, '14467': 1268, '14741': 1841, '14749': 714, '14813': 1763, '14967': 1394, '15225': 919, '15315': 1278, '15451': 1277, '15583': 1305, '15809': 2021, '16049': 959, '16157': 213, '16444': 288, '16498': 3352, '16512': 476, '1

In [16]:
def get_transfer_prob(vs):
    neighbours = vs.keys()
    total_weight = sum(vs.values())
    
    probs = [vs[k]/total_weight for k in neighbours]
    try:
        assert( abs(1.0 - sum(probs)) < 0.000000001 )
    except:
        print(sum(probs))
        print(vs)
    
    return {
        'neighbours': list(neighbours),
        'probs': probs
    }

In [17]:
# 3. build prob transfer matrix, for each key, each value <=> probability in total weights
transfer_probs = {k: get_transfer_prob(v) for k, v in matrix.items()}
print(f'transfer probs built. {len(transfer_probs)} entrances')

transfer probs built. 8265 entrances


In [18]:
transfer_probs['12049']

{'neighbours': ['356',
  '1195',
  '1535',
  '1575',
  '1604',
  '1840',
  '2167',
  '2904',
  '2993',
  '3455',
  '3712',
  '4181',
  '4214',
  '4224',
  '4654',
  '6201',
  '6213',
  '6324',
  '6347',
  '6547',
  '6702',
  '7054',
  '7592',
  '7593',
  '8074',
  '8516',
  '8525',
  '8536',
  '8769',
  '8841',
  '8937',
  '9041',
  '9513',
  '9919',
  '10049',
  '10080',
  '10213',
  '10620',
  '10719',
  '10790',
  '10793',
  '11319',
  '11617',
  '11633',
  '11703',
  '11743',
  '11757',
  '11933',
  '13659',
  '13759',
  '14075',
  '14227',
  '14345',
  '14467',
  '14741',
  '14749',
  '14813',
  '14967',
  '15225',
  '15315',
  '15451',
  '15583',
  '15809',
  '16049',
  '16157',
  '16444',
  '16498',
  '16512',
  '16668',
  '16706',
  '17651',
  '17741',
  '18857',
  '1',
  '15',
  '16',
  '20',
  '60',
  '68',
  '71',
  '73',
  '180',
  '181',
  '189',
  '195',
  '198',
  '205',
  '223',
  '225',
  '226',
  '237',
  '246',
  '355',
  '390',
  '469',
  '481',
  '502',
  '527',
  

In [19]:
# 4. entrance
counts = {k: sum(neibors.values()) for k, neibors in matrix.items()}
entrance_items = list(transfer_probs.keys())    ## all keys
total_count = sum(counts.values())  ## all values => 加在一起
entrance_probs = [counts[k] / total_count for k in entrance_items]  ## each entrance probability
print(counts['6030'])   ## each entrance key: sum of values

print(total_count)
print(entrance_probs[5])
# print(entrance_items)
print('entrance probs built.')

296925
620556990
0.0015106380479253001
entrance probs built.


In [20]:
## DEEP WALK
rng = np.random.default_rng()
## rng.choice(entrance_items, 1, p=entrance_probs): 
## This is a function call that randomly selects one element from entrance_items based on the probabilities provided in entrance_probs.
def one_random_walk(length, entrance_items, entrance_probs, transfer_probs):
    start_vertex = rng.choice(entrance_items, 1, p=entrance_probs)[0]
    path = [str(start_vertex)]
    
    curr_vertex = start_vertex

    for _ in range(length):
        if curr_vertex not in transfer_probs:
            print(f"bad vertex {curr_vertex}")
            break
            
        neighbours = transfer_probs[curr_vertex]['neighbours']
        trans_prob = transfer_probs[curr_vertex]['probs']
        
        try:
            next_vertex = rng.choice(neighbours, 1, p=trans_prob)[0]
            path.append(str(next_vertex))

            curr_vertex = next_vertex
        except Exception as e:
            print(curr_vertex)
            print(e)
            break
        
    return path

In [21]:
n = 500
samples = [one_random_walk(20, entrance_items, entrance_probs, transfer_probs) for _ in range(n)]

In [22]:
samples[:4]

[['1887',
  '31240',
  '210',
  '11433',
  '15689',
  '18679',
  '1172',
  '2899',
  '5118',
  '1941',
  '15417',
  '2685',
  '2025',
  '1543',
  '16742',
  '7724',
  '2025',
  '225',
  '2476',
  '2001',
  '20853'],
 ['3901',
  '2904',
  '287',
  '23283',
  '11123',
  '1770',
  '6024',
  '1562',
  '1827',
  '189',
  '2490',
  '1562',
  '7579',
  '10278',
  '14349',
  '457',
  '9690',
  '760',
  '16011',
  '9917',
  '1686'],
 ['4192',
  '11617',
  '16904',
  '17265',
  '10687',
  '468',
  '5081',
  '31608',
  '30279',
  '11979',
  '20745',
  '6922',
  '7661',
  '3269',
  '12531',
  '18095',
  '22835',
  '7465',
  '14353',
  '11061',
  '4898'],
 ['28999',
  '28283',
  '1210',
  '3783',
  '14131',
  '5356',
  '3653',
  '195',
  '225',
  '1668',
  '264',
  '2026',
  '287',
  '10578',
  '298',
  '18139',
  '14511',
  '2889',
  '1535',
  '853',
  '5204']]

In [ ]:
#### item2

In [23]:
sample_df = spark.createDataFrame([[row] for row in samples], ['anime_ids'])

In [24]:
sample_df.show()

+--------------------+
|           anime_ids|
+--------------------+
|[1887, 31240, 210...|
|[3901, 2904, 287,...|
|[4192, 11617, 169...|
|[28999, 28283, 12...|
|[969, 10521, 5150...|
|[27655, 853, 5630...|
|[21431, 21863, 56...|
|[9471, 5204, 1531...|
|[10162, 2104, 350...|
|[5162, 23273, 120...|
|[1723, 32108, 158...|
|[517, 185, 9941, ...|
|[8841, 13601, 646...|
|[1132, 4970, 72, ...|
|[21327, 20903, 14...|
|[893, 1889, 31138...|
|[587, 24415, 8425...|
|[1210, 4472, 1943...|
|[6201, 272, 6336,...|
|[31704, 20431, 19...|
+--------------------+
only showing top 20 rows



In [25]:
from pyspark.ml.feature import Word2Vec


In [28]:
item2vec = Word2Vec(vectorSize=5, maxIter=2, windowSize=15)
item2vec.setInputCol('anime_ids')
item2vec.setOutputCol('anime_ids_vec')

Word2Vec_53a463bf30c6

In [29]:
model = item2vec.fit(sample_df)

24/08/21 21:13:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [30]:
rec = model.findSynonyms('20', 10)

In [31]:
rec.show()

+-----+------------------+
| word|        similarity|
+-----+------------------+
| 2951|0.9735549092292786|
| 2985|0.9681600332260132|
|12049|0.9660470485687256|
|28297|0.9584445953369141|
| 5365|0.9481610059738159|
|  196|0.9320666790008545|
|22297|0.9308104515075684|
|27899|0.9275564551353455|
|  813|0.9270269870758057|
| 9989|0.9256731867790222|
+-----+------------------+

